In [8]:
# fast_png_to_webp_one.py
import os, io, math
from PIL import Image

# ---- CONFIG ----
TARGET_BYTES = int(120 * 1024)   # <= 120 KB
PREF_MIN_SIDE = 600              # prefer to stay above this
ABS_MIN_SIDE  = 256              # absolute lower bound
MAX_START_SIDE = 2048            # pre-shrink huge inputs to this max side
METHOD = 4                       # 0..6: lower is faster
MAX_QUALITY_TRIES = 5            # cap re-encodes per size level
REDUCING_GAP = 2.0               # speeds thumbnail downscaling
# ----------------

def encode_webp(im: Image.Image, quality: int) -> bytes:
    buf = io.BytesIO()
    im.save(
        buf,
        format="WEBP",
        quality=quality,
        method=METHOD,
        lossless=False,
        exact=False,            # faster; keep True only if you need exact transparent color
        alpha_quality=90        # reduce a bit for speed/size balance
        # omit icc/exif for speed unless you need them
    )
    return buf.getvalue()

def size_in_bytes(data: bytes) -> int:
    return len(data)

def good_start_quality(megapixels: float, target_kb: float) -> int:
    """
    Heuristic: pick a starting quality from image size and target size.
    Bigger images or smaller targets -> lower starting Q.
    """
    # Simple rule-of-thumb curve
    if megapixels <= 0.5:
        base = 85
    elif megapixels <= 1.5:
        base = 80
    elif megapixels <= 3:
        base = 75
    else:
        base = 70

    # Tighten for stricter targets
    if target_kb < 90:  base -= 5
    if target_kb < 70:  base -= 5
    if target_kb < 50:  base -= 5
    return max(35, min(92, base))

def try_qualities_fast(im: Image.Image, target_bytes: int, start_q: int) -> tuple[bytes, int]:
    """
    Try a small sequence of qualities around start_q, adjusting based on result.
    At most MAX_QUALITY_TRIES encodes.
    """
    q = start_q
    best_bytes = None
    best_q = q

    # small bounded search: q, q-10/q+10, then refine ±5, etc.
    steps = [0, -10, +10, -5, +5][:MAX_QUALITY_TRIES]
    for step in steps:
        cand_q = max(35, min(92, q + step))
        data = encode_webp(im, cand_q)
        if best_bytes is None or size_in_bytes(data) < size_in_bytes(best_bytes):
            best_bytes, best_q = data, cand_q
        if size_in_bytes(data) <= target_bytes:
            return data, cand_q
    # If none fit, return the smallest so far
    return best_bytes, best_q

def shrink_once(im: Image.Image, factor: float = 0.85) -> Image.Image:
    w, h = im.size
    nw = max(int(w * factor), 1)
    nh = max(int(h * factor), 1)
    # thumbnail is faster with reducing_gap, keeps aspect, and uses good resampling
    im2 = im.copy()
    im2.thumbnail((nw, nh), Image.LANCZOS, reducing_gap=REDUCING_GAP)
    return im2

def ensure_max_side(im: Image.Image, max_side: int) -> Image.Image:
    if max(im.size) <= max_side:
        return im
    im2 = im.copy()
    im2.thumbnail((max_side, max_side), Image.LANCZOS, reducing_gap=REDUCING_GAP)
    return im2

def save_quick_under_cap(im: Image.Image, out_path: str, target_bytes: int) -> tuple[int, tuple[int,int]]:
    im = im.convert("RGBA")

    # Pre-shrink huge sources once
    im = ensure_max_side(im, MAX_START_SIDE)

    # 1) Try at current size with a good starting quality
    mp = (im.size[0] * im.size[1]) / 1_000_000.0
    start_q = good_start_quality(mp, target_bytes/1024)
    data, used_q = try_qualities_fast(im, target_bytes, start_q)
    if size_in_bytes(data) <= target_bytes:
        with open(out_path, "wb") as f: f.write(data)
        return len(data), im.size

    # 2) If too big, shrink in a few quick steps (prefer not to go below PREF_MIN_SIDE)
    while max(im.size) > PREF_MIN_SIDE:
        im = shrink_once(im)
        data, used_q = try_qualities_fast(im, target_bytes, start_q=max(35, used_q-5))
        if size_in_bytes(data) <= target_bytes:
            with open(out_path, "wb") as f: f.write(data)
            return len(data), im.size

    # 3) Last-resort: allow going below PREF_MIN_SIDE down to ABS_MIN_SIDE
    while max(im.size) > ABS_MIN_SIDE:
        im = shrink_once(im)
        data, used_q = try_qualities_fast(im, target_bytes, start_q=max(35, used_q-5))
        if size_in_bytes(data) <= target_bytes:
            with open(out_path, "wb") as f: f.write(data)
            return len(data), im.size

    # 4) Write the best we got (might still be slightly over if the image is extremely complex)
    with open(out_path, "wb") as f: f.write(data)
    return len(data), im.size

def main():
    in_path = input("Paste PNG path: ").strip().strip('"').strip("'")
    if not os.path.exists(in_path):
        print(f"[ERROR] Not found: {in_path}")
        return

    out_path = os.path.splitext(in_path)[0] + ".webp"
    with Image.open(in_path) as img:
        bytes_out, (w, h) = save_quick_under_cap(img, out_path, TARGET_BYTES)

    print(f"[DONE] -> {out_path}")
    print(f"       Final: {bytes_out/1024:.1f} KB | {w}x{h} | Target: {TARGET_BYTES/1024:.0f} KB | method={METHOD}")

if __name__ == "__main__":
    main()


[DONE] -> C:\Users\jzlot\Downloads\joeshero.webp
       Final: 113.2 KB | 1530x1020 | Target: 120 KB | method=4
